<a href="https://colab.research.google.com/github/tubagokhan/RegNLPDataset/blob/main/RIRAGEvaluationMetricFull.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown 1BB21XL5geQ0Uw6gk0_uRFk6tthPQ0vSh
!gdown 1BGb3EYmhmTMVx3qTihCBpUrJ3WO2lM8H

Downloading...
From: https://drive.google.com/uc?id=1BB21XL5geQ0Uw6gk0_uRFk6tthPQ0vSh
To: /content/ObligationClassificationDataset.json
100% 545k/545k [00:00<00:00, 114MB/s]
Downloading...
From: https://drive.google.com/uc?id=1BGb3EYmhmTMVx3qTihCBpUrJ3WO2lM8H
To: /content/selected_test_samples.json
100% 135k/135k [00:00<00:00, 120MB/s]


In [2]:
import json
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Check if CUDA is available and use it if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Step 1: Load and preprocess the data
json_path = "./ObligationClassificationDataset.json"
with open(json_path, 'r') as file:
    data = json.load(file)

texts = [item['Text'] for item in data]
labels = [1 if item['Obligation'] else 0 for item in data]  # Converting True/False to 1/0

# Step 2: Tokenization using LegalBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased')

class ObligationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Splitting data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(texts, labels, test_size=0.2, random_state=42)

train_dataset = ObligationDataset(X_train, y_train, tokenizer)
val_dataset = ObligationDataset(X_val, y_val, tokenizer)

# Step 3: Fine-tuning LegalBERT for sequence classification
model = AutoModelForSequenceClassification.from_pretrained('nlpaueb/legal-bert-base-uncased', num_labels=2)
model.to(device)  # Move model to the GPU

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Step 4: Train the model
trainer.train()

# Step 5: Evaluate the model
trainer.evaluate()

# Step 6: Save the model and tokenizer for future use
model.save_pretrained('./obligation-classifier-legalbert')
tokenizer.save_pretrained('./obligation-classifier-legalbert')

print("Model fine-tuning and evaluation completed.")


Using device: cuda


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.477400,0.384716,0.850000,0.887070,0.813814,0.974820
2,0.051200,0.080557,0.980435,0.983957,0.975265,0.992806
3,0.066200,0.025580,0.991304,0.992806,0.992806,0.992806
4,0.000200,0.034434,0.993478,0.994595,0.996390,0.992806
5,0.020800,0.100885,0.984783,0.987250,1.000000,0.974820
6,0.047800,0.057023,0.991304,0.992754,1.000000,0.985612


Model fine-tuning and evaluation completed.


In [3]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.8 MB/s eta 0:00:00


In [67]:
import json
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from nltk.tokenize import sent_tokenize as sent_tokenize_uncached
import nltk
from functools import cache
import tqdm

nltk.download('punkt')

# Set up device for torch operations
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load the tokenizer and model for obligation detection
model_name = './obligation-classifier-legalbert'
obligation_tokenizer = AutoTokenizer.from_pretrained(model_name)
obligation_model = AutoModelForSequenceClassification.from_pretrained(model_name)
obligation_model.to(device)
obligation_model.eval()

# Load NLI model and tokenizer for obligation coverage using Microsoft's model
coverage_nli_model = pipeline("text-classification", model="microsoft/deberta-large-mnli", device=device)

# Load NLI model and tokenizer for entailment and contradiction checks
nli_tokenizer = AutoTokenizer.from_pretrained('cross-encoder/nli-deberta-v3-xsmall')
nli_model = AutoModelForSequenceClassification.from_pretrained('cross-encoder/nli-deberta-v3-xsmall')
nli_model.to(device)
nli_model.eval()

# Define a cached version of sentence tokenization
@cache
def sent_tokenize(passage: str):
  return sent_tokenize_uncached(passage)

def softmax(logits):
    e_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
    return e_logits / np.sum(e_logits, axis=1, keepdims=True)

def get_nli_probabilities(premises, hypotheses):
    features = tokenizer(premises, hypotheses, padding=True, truncation=True, return_tensors="pt").to("cuda")
    nli_model.eval()
    with torch.no_grad():
        logits = nli_model(**features).logits.cpu().numpy()
    probabilities = softmax(logits)
    return probabilities

def get_nli_matrix(passages, answers):
    print(f"{len(passages)} passages and {len(answers)} answers.")
    entailment_matrix = np.zeros((len(passages), len(answers)))
    contradiction_matrix = np.zeros((len(passages), len(answers)))

    batch_size = 16
    for i, pas in enumerate(tqdm.tqdm(passages)):
      for b in range(0, len(answers), batch_size):
        e = b + batch_size
        probs = get_nli_probabilities([pas] * len(answers[b:e]), answers[b:e])  # Get NLI probabilities
        entailment_matrix[i, b:e] = probs[:, 1]
        contradiction_matrix[i, b:e] = probs[:, 0]
    return entailment_matrix, contradiction_matrix

def calculate_scores_from_matrix(nli_matrix, score_type='entailment'):
    if nli_matrix.size == 0:
        print("Warning: NLI matrix is empty. Returning default score of 0.")
        return 0.0  # or some other default score or handling as appropriate for your use case

    if score_type == 'entailment':
        reduced_vector = np.max(nli_matrix, axis=0)
    elif score_type == 'contradiction':
        reduced_vector = np.min(nli_matrix, axis=0)
    score = np.round(np.mean(reduced_vector), 5)
    return score

def calculate_obligation_coverage_score(passages, answers):
    obligation_sentences_source = [sent for passage in passages for sent in sent_tokenize(passage)]
    obligation_sentences_answer = [sent for answer in answers for sent in sent_tokenize(answer)]
    covered_count = 0

    for obligation in obligation_sentences_source:
        obligation_covered = False
        for answer_sentence in obligation_sentences_answer:
            nli_result = coverage_nli_model(f"{answer_sentence} [SEP] {obligation}")
            if nli_result[0]['label'].lower() == 'entailment' and nli_result[0]['score'] > 0.7:
                covered_count += 1
                obligation_covered = True
                break


    coverage_score = covered_count / len(obligation_sentences_source) if obligation_sentences_source else 0
    return coverage_score

def calculate_final_composite_score(passages, answers):
    passage_sentences = [sent for passage in passages for sent in sent_tokenize(passage)]
    answer_sentences = [sent for answer in answers for sent in sent_tokenize(answer)]
    entailment_matrix, contradiction_matrix = get_nli_matrix(passage_sentences, answer_sentences)
    entailment_score = calculate_scores_from_matrix(entailment_matrix, 'entailment')
    contradiction_score = calculate_scores_from_matrix(contradiction_matrix, 'contradiction')
    obligation_coverage_score = calculate_obligation_coverage_score(passages, answers)
    print(f"Entailment Score: {entailment_score}")
    print(f"Contradiction Score: {contradiction_score}")
    print(f"Obligation Coverage Score: {obligation_coverage_score}")


    # New formula: (O + E - C + 1) / 3
    composite_score = (obligation_coverage_score + entailment_score - contradiction_score + 1) / 3
    print(f"Final Composite Score: {composite_score}")
    return np.round(composite_score, 5)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [74]:
test_data=[
    {
    "QuestionID": "1",
    "Question": "What can I do sometimes",
    "RetrievedPassage": "The proposed merger aims to combine resources. While this move is expected to result in significant market influence and revenue increase, it also brings risks such as integration challenges, potential layoffs, and regulatory scrutiny.",
    "Answer": "to be or not to be. Is there anybody there? You may carefully apply the rules."
  },
  {
    "QuestionID": "1",
    "Question": "What are the potential risks and benefits associated with the new merger proposal?",
    "RetrievedPassage": "The proposed merger aims to combine resources of two leading tech firms to drive innovation and market expansion. While this move is expected to result in significant market influence and revenue increase, it also brings risks such as integration challenges, potential layoffs, and regulatory scrutiny.",
    "Answer": "The merger is poised to significantly enhance our market position by pooling resources and expertise. However, we must navigate challenges such as the harmonization of corporate cultures and potential regulatory issues that may arise."
  },
  {
    "QuestionID": "2",
    "Question": "How will the new environmental regulation impact our manufacturing processes?",
    "RetrievedPassage": "The new environmental regulation mandates reductions in carbon emissions and requires the use of cleaner technologies. This will necessitate a shift in our manufacturing processes, involving substantial investment in new technology and potential restructuring of our facilities.",
    "Answer": "To comply with the new regulation, we will be overhauling our current manufacturing processes, introducing energy-efficient technologies, and potentially facing short-term disruptions but ultimately improving our sustainability and compliance."
  },
  {
    "QuestionID": "3",
    "Question": "What steps are being taken to improve employee satisfaction and retention?",
    "RetrievedPassage": "In response to recent surveys indicating low employee satisfaction, the company plans to implement several initiatives. These include increasing compensation, offering more flexible working conditions, enhancing career development opportunities, and improving workplace culture.",
    "Answer": "We are committed to improving employee satisfaction through comprehensive measures including competitive salary adjustments, flexible work policies, expanded professional development, and fostering a more inclusive and supportive work environment."
  },
  {
    "QuestionID": "4",
    "Question": "Can you outline the new product development plan for the next year?",
    "RetrievedPassage": "The product development plan for the next year focuses on leveraging advanced technologies to introduce a new line of smart devices. This plan includes phases of market research, prototype development, user testing, and a final market launch aimed for the fourth quarter.",
    "Answer": "Our strategy involves a step-by-step rollout of new smart devices, starting with intensive market research, moving through several stages of design refinement and user testing, and culminating in a targeted launch by year-end to capture holiday sales."
  },
  {
    "QuestionID": "5",
    "Question": "What are the primary objectives of our latest strategic initiative?",
    "RetrievedPassage": "The primary objectives of our latest strategic initiative include increasing global market share, diversifying our product line to include eco-friendly options, and enhancing digital transformation across all departments to improve operational efficiency.",
    "Answer": "Our strategic initiative aims to position us as a leader in the global market by expanding our reach, introducing sustainable products that meet modern environmental standards, and adopting cutting-edge digital solutions to streamline operations and reduce costs."
  },
  {
    "QuestionID": "6",
    "Question": "What are the challenges facing the company's expansion into Asian markets?",
    "RetrievedPassage": "Expanding into Asian markets presents several challenges including navigating diverse regulatory landscapes, understanding cultural nuances in consumer behavior, and competing with established local brands. Additionally, logistical complexities and the need for local partnerships complicate market entry.",
    "Answer": "Our expansion strategy must address the intricate regulatory environments and cultural diversity in Asia. We plan to forge strong local partnerships, adapt our marketing strategies to align with consumer preferences, and build a logistics network that supports efficient distribution."
  },
  {
    "QuestionID": "7",
    "Question": "Describe the cybersecurity measures in place to protect client data.",
    "RetrievedPassage": "Our cybersecurity measures include multi-layered security protocols, regular system audits, and compliance with international data protection regulations. We utilize advanced encryption techniques, continuous monitoring systems, and have implemented strict access controls and training programs for all staff.",
    "Answer": "We ensure the highest level of data security through robust encryption standards, ongoing surveillance of our IT infrastructure, and adherence to global best practices in data protection. Our proactive approach involves regular audits and updates to our security protocols to guard against emerging threats."
  },
  {
    "QuestionID": "8",
    "Question": "How do current economic conditions affect our business strategy?",
    "RetrievedPassage": "The current economic conditions, characterized by fluctuating market trends and consumer spending patterns, necessitate a flexible approach to our business strategy. This involves adapting our product offerings and pricing models to meet changing demands, optimizing supply chains to reduce costs, and enhancing our online presence.",
    "Answer": "In response to volatile economic conditions, we are adjusting our business model to be more agile. This includes revising our pricing strategies, expanding our digital marketing efforts, and streamlining operations to maintain profitability despite economic pressures."
  },
  {
    "QuestionID": "9",
    "Question": "What improvements have been made in customer service this year?",
    "RetrievedPassage": "This year, we've made significant improvements in customer service by integrating AI technology into our support systems, training staff extensively, and expanding our online service platforms to ensure 24/7 availability. These initiatives have been aimed at reducing response times and enhancing user satisfaction.",
    "Answer": "Our customer service has been transformed through the adoption of AI-driven support tools that offer real-time assistance and predictive problem-solving. Enhanced training programs for our team ensure a high standard of service, while expanded online platforms provide constant accessibility and support."
  },
  {
    "QuestionID": "10",
    "Question": "Explain the financial implications of the recent merger.",
    "RetrievedPassage": "The recent merger is expected to streamline operations and achieve cost synergies by combining resources, which should result in increased profitability. However, it also involves significant upfront costs related to integration, potential redundancy payouts, and increased debt load.",
    "Answer": "While the merger presents initial financial challenges, including integration costs and the assumption of some debt, it positions us for greater long-term profitability through operational efficiencies and a broader market presence. Cost synergies should start to be realized within the first two years post-merger."
  }
]


In [76]:
# Load the selected test samples JSON
input_file_path = '/content/GoldenQuestionAnswerforEvaluationMetric.json'
with open(input_file_path, 'r') as file:
    test_data = json.load(file)

# Prepare the data
composite_scores = []
entailment_scores = []
contradiction_scores = []
obligation_coverage_scores = []

for item in test_data:
    question = [item['Question']]
    passages = [item['RetrievedPassage']]
    answers = [item['Answer']]
    print(f"Question: {question}")

    # Calculate and store scores
    passage_sentences = [sent for passage in passages for sent in sent_tokenize(passage)]
    answer_sentences = [sent for answer in answers for sent in sent_tokenize(answer)]
    entailment_matrix, contradiction_matrix = get_nli_matrix(passage_sentences, answer_sentences)
    entailment_score = calculate_scores_from_matrix(entailment_matrix, 'entailment')
    contradiction_score = calculate_scores_from_matrix(contradiction_matrix, 'contradiction')
    obligation_coverage_score = calculate_obligation_coverage_score(passages, answers)
    final_composite_score = (obligation_coverage_score + entailment_score - contradiction_score + 1) / 3

    # Append to respective lists
    entailment_scores.append(entailment_score)
    contradiction_scores.append(contradiction_score)
    obligation_coverage_scores.append(obligation_coverage_score)
    composite_scores.append(final_composite_score)

    print("\n")

# Calculate averages
avg_entailment = np.mean(entailment_scores)
avg_contradiction = np.mean(contradiction_scores)
avg_obligation_coverage = np.mean(obligation_coverage_scores)
avg_composite = np.mean(composite_scores)

print("\n")
print("Average Entailment Score:", avg_entailment)
print("Average Contradiction Score:", avg_contradiction)
print("Average Obligation Coverage Score:", avg_obligation_coverage)
print("Average Final Composite Score:", avg_composite)


Question: ["Do we regulate a CFD based upon a spot trade 'rolling a spot forex contract'?"]
3 passages and 4 answers.


100%|██████████| 3/3 [00:00<00:00, 26.77it/s]




Question: ['Does a set of threshold conditions set in a written undertaking be legally binding?']
6 passages and 7 answers.


100%|██████████| 6/6 [00:00<00:00, 24.42it/s]




Question: ['Can the FSRA exercise its powers to object to a change in control based on the unsuitability of the Controller?']
11 passages and 12 answers.


...



Question: ['What are the provisions in FSRA in case a request from a company licensed by your authority, to acquire 30% or more of a non-licensed company (does not have a license from your authority) that is located outside your jurisdiction or outside the UAE?']
11 passages and 17 answers.


100%|██████████| 11/11 [00:02<00:00,  4.42it/s]




Question: ['Do the Waiver/Modification application Notices have to go through PWMC? Or an approval from the CEO would be considered enough?']
3 passages and 4 answers.


100%|██████████| 3/3 [00:00<00:00, 38.70it/s]






Average Entailment Score: 0.8372866666666667
Average Contradiction Score: 0.06893619047619048
Average Obligation Coverage Score: 1.0
Average Final Composite Score: 0.9227834920634921


In [77]:
# Load the selected test samples JSON
input_file_path = '/content/selected_test_samples.json'
with open(input_file_path, 'r') as file:
    test_data = json.load(file)

# Prepare the data
composite_scores = []
entailment_scores = []
contradiction_scores = []
obligation_coverage_scores = []

for item in test_data:
    question = [item['Question']]
    passages = [item['RetrievedPassage']]
    answers = [item['Answer']]
    print(f"Question: {question}")

    # Calculate and store scores
    passage_sentences = [sent for passage in passages for sent in sent_tokenize(passage)]
    answer_sentences = [sent for answer in answers for sent in sent_tokenize(answer)]
    entailment_matrix, contradiction_matrix = get_nli_matrix(passage_sentences, answer_sentences)
    entailment_score = calculate_scores_from_matrix(entailment_matrix, 'entailment')
    contradiction_score = calculate_scores_from_matrix(contradiction_matrix, 'contradiction')
    obligation_coverage_score = calculate_obligation_coverage_score(passages, answers)
    final_composite_score = (obligation_coverage_score + entailment_score - contradiction_score + 1) / 3

    # Append to respective lists
    entailment_scores.append(entailment_score)
    contradiction_scores.append(contradiction_score)
    obligation_coverage_scores.append(obligation_coverage_score)
    composite_scores.append(final_composite_score)

    print("\n")

# Calculate averages
avg_entailment = np.mean(entailment_scores)
avg_contradiction = np.mean(contradiction_scores)
avg_obligation_coverage = np.mean(obligation_coverage_scores)
avg_composite = np.mean(composite_scores)

print("\n")
print("Average Entailment Score:", avg_entailment)
print("Average Contradiction Score:", avg_contradiction)
print("Average Obligation Coverage Score:", avg_obligation_coverage)
print("Average Final Composite Score:", avg_composite)


Question: ['Are client classification requirements applicable to an Authorised Person operating a Representative Office when engaging with a Person regarding Regulated Activities or Specified Investments?']
2 passages and 3 answers.


100%|██████████| 2/2 [00:00<00:00, 37.72it/s]




Question: ['When categorizing Clients, what are the two Client categories that an Authorised Person must consider?']
2 passages and 2 answers.


100%|██████████| 2/2 [00:00<00:00, 40.06it/s]




Question: ['How does an Authorised Person determine the appropriate Client category for each Client?']
2 passages and 8 answers.


...



Question: ["Under what conditions is it permissible for an Authorised Person to conduct a Regulated Activity with a Client on amended terms without giving the fourteen days' notice as stated in the Client Agreement?"]
1 passages and 2 answers.


100%|██████████| 1/1 [00:00<00:00, 34.26it/s]




Question: ['What are the obligations of an Authorised Person under Rule 3.4.2(b) regarding recommending Specified Investments or carrying out Regulated Activities for a Client?']
2 passages and 5 answers.


100%|██████████| 2/2 [00:00<00:00, 35.29it/s]






Average Entailment Score: 0.7024035
Average Contradiction Score: 0.1372604
Average Obligation Coverage Score: 0.34450000000000003
Average Final Composite Score: 0.6365476999999999
